In [1]:
from gurobipy import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import shapefile as shp
from collections import defaultdict

### Set Parameters

In [2]:
M = 10000
maxOpenD = 50
maxOpenA = 20
maxDistance = 10000

maxDist_D = 150
maxDist_A = 350

#maxPopD- max capacity of each district court
#maxPopA - max capacity of each appeals court

#xS,yS 
#xD,yD
#xA,yA

#langS
#langD
#langA


#param d{i in Settlements,j in DistrictCourts} 
#	:= sqrt( (xS[i]-xD[j])^2 + (yS[i]-yD[j])^2); #distance between settlement and district court
#param a{i in Settlements,k in AppealsCourts} 
#	:= sqrt( (xA[k]-xS[i])^2 + (yA[k]-yS[i])^2); #distance between district court and appeals court	

### Extract Data

In [3]:
#Get Settlement List
Settlements = pd.read_csv("afg_ppl_settlement_pnt.csv",sep=",")
Settlements = Settlements[[0,10,11]]
S = Settlements.shape[0]
Settlement_List = Settlements['OBJECTID'].tolist()

#Get District Court List
Districts = pd.read_csv("District_Courts.csv",sep=",")
Districts = Districts[[1,4,5]]
D = Districts.shape[0]
District_List = Districts['DIST_CODE'].tolist()

#Get Appeals Court List
Appeals = pd.read_csv("Appeals_Courts.csv",sep=",")
Appeals = Appeals[[1,4,5]]
A = Appeals.shape[0]
Appeals_List = Appeals['PROV_CODE'].tolist()

#### Create Data Subset for Settlements

#### ------------------------------------------------

In [4]:
Settlements = Settlements.sample(frac = 0.01, replace = False)
S = Settlements.shape[0]
Settlement_List = Settlements['OBJECTID'].tolist()

####  ------------------------------------------------

In [5]:
#Create Dictionaries
Settlement_Dict = Settlements.set_index('OBJECTID').T.to_dict('list')
District_Dict = Districts.set_index('DIST_CODE').T.to_dict('list')
Appeals_Dict = Appeals.set_index('PROV_CODE').T.to_dict('list')

In [6]:
#Create Dictionaries for District Courthouse Distances

Dist_D = {}

R = 6371e3


for d in District_List:
    d_lon = District_Dict[d][0]
    d_lat = District_Dict[d][1]
    φ2 = np.radians(d_lat)


    for s in Settlement_List:
        s_lon = Settlement_Dict[s][1]
        s_lat = Settlement_Dict[s][0]
        φ1 = np.radians(s_lat)
        
        #Distance to District Court
        Δφ = np.radians(d_lat - s_lat)
        Δλ = np.radians(d_lon - s_lon)
        a = np.sin(Δφ/2) * np.sin(Δφ/2) + np.cos(φ1) * np.cos(φ2) * np.sin(Δλ/2) * np.sin(Δλ/2)
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        
        Dist_D[s,d] = (R * c)/1000
    

In [7]:
#Create Dictionaries for Appeals Courthouse Distances

Dist_A = {}

R = 6371e3

for a in Appeals_List:
    a_lon = Appeals_Dict[a][0]
    a_lat = Appeals_Dict[a][1]
    φ3 = np.radians(a_lat)

    for s in Settlement_List:
        s_lon = Settlement_Dict[s][1]
        s_lat = Settlement_Dict[s][0]
        φ1 = np.radians(s_lat)
        
        #Distance to District Court
        Δφ = np.radians(a_lat - s_lat)
        Δλ = np.radians(a_lon - s_lon)
        a1 = np.sin(Δφ/2) * np.sin(Δφ/2) + np.cos(φ1) * np.cos(φ3) * np.sin(Δλ/2) * np.sin(Δλ/2)
        c = 2 * np.arctan2(np.sqrt(a1), np.sqrt(1-a1))
        
        Dist_A[s,a] = (R * c)/1000

##### Helper function

In [8]:
# Return value of variable
def VarVal(var):
    if (type(var) == gurobipy.Var):
        val = var.X
    else:
        val = 0
    return val

# ===========================================

### Create Model - LP

In [9]:
#Create Model
LP = Model("Afg_LP")

#Suppress output
LP.Params.OutputFlag = 0

### Create Variables - LP

##### d_ij - LP

In [10]:
# Create d_i_j LP variables
d_LP = {}
for i in Settlement_List:
    d_LP[i] = {}
    for j in District_List:
        if Dist_D[i,j] < maxDist_D:
            d_LP[i][j] = LP.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name='d_LP_%s_%s' % (i, j))
        else:
            d_LP[i][j] = 0
LP.update()

In [11]:
# Create d_i_j LP transpose dictionary 
d_t_LP = defaultdict(dict)
for key, val in d_LP.items():
    for subkey, subval in val.items():
        d_t_LP[subkey][key] = subval

##### a_ik - LP

In [12]:
 # Create a_i_k LP variables
a_LP = {}
for i in Settlement_List:
    a_LP[i] = {}
    for k in Appeals_List:
        if Dist_A[i,k] < maxDist_A:
            a_LP[i][k] = LP.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name='a_LP_%s_%s' % (i, k))
        else:
            a_LP[i][k] = 0
LP.update()

In [13]:
# Create a_i_k LP transpose dictionary 
a_t_LP = defaultdict(dict)
for key, val in a_LP.items():
    for subkey, subval in val.items():
        a_t_LP[subkey][key] = subval

##### c_jk - LP

In [14]:
# Create c_j_k LP variables
c_LP = {}
for j in District_List:
    c_LP[j] = {}
    for k in Appeals_List:
        c_LP[j][k] = LP.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name='c_LP_%s_%s' % (j, k))
LP.update()

In [15]:
# Create c_j_k LP transpose dictionary 
c_t_LP = defaultdict(dict)
for key, val in c_LP.items():
    for subkey, subval in val.items():
        c_t_LP[subkey][key] = subval

##### openD - LP

In [16]:
#Create openD LP variables
openD_LP = {}
for j in District_List:
    openD_LP[j] = LP.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name='openD_LP_%s' % (j))
LP.update()

##### openA - LP

In [17]:
#Create openA LP variables
openA_LP = {}
for k in Appeals_List:
    openA_LP[k] = LP.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name='openA_LP_%s' % (k))
LP.update()

### Create Constraints - LP

#### D_ij row sums and column sums - LP

In [18]:
#One S -> D Assignment LP
for i in Settlement_List:
    LP.addConstr(quicksum(d_LP[i][j] for j in d_LP[i]) == 1)
LP.update()

In [19]:
#Maximum District Courts Open LP
for i in Settlement_List:
    for j in District_List:
        if (type(d_LP[i][j]) == gurobipy.Var):
            LP.addConstr(d_LP[i][j] <= openD_LP[j])
LP.update()

#### A_ik row sums and column sums - LP

In [20]:
#One S -> A Assignment LP
for i in Settlement_List:
    LP.addConstr(quicksum(a_LP[i][k] for k in a_LP[i]) == 1)
LP.update()

In [21]:
#Maximum Appeals Courts Open LP
for i in Settlement_List:
    for k in Appeals_List:
        if (type(a_LP[i][k]) == gurobipy.Var):
            LP.addConstr(a_LP[i][k] <= openA_LP[k])
LP.update()

#### C_jk row sums and column sums - LP

In [22]:
#One D -> A Assignment LP
for j in District_List:
    LP.addConstr(quicksum(c_LP[j][k] for k in c_LP[j]) == openD_LP[j])
LP.update()

In [23]:
#C constraint LP
for j in District_List:
    for k in Appeals_List:
        LP.addConstr(c_LP[j][k] <= openA_LP[k])
LP.update()

##### Max Open Courthouse Constraints - LP

In [24]:
LP.addConstr(quicksum(openD_LP[j] for j in District_List) <= maxOpenD)
LP.addConstr(quicksum(openA_LP[k] for k in Appeals_List) <= maxOpenA)

LP.update()

### Set Objective Function - LP

#### D_ij and A_ik - LP

In [25]:
LP.setObjective(
        quicksum(   quicksum(   Dist_D[i,j]*d_t_LP[j][i] for i in d_t_LP[j])    for j in District_List) + \
        quicksum(   quicksum(   Dist_A[i,k]*a_t_LP[k][i] for i in a_t_LP[k])    for k in Appeals_List), GRB.MINIMIZE)
LP.update()

### Optimize

In [26]:
violated_LP = 1

while violated_LP > 0:
        
    print("LP Optimize")
    LP.optimize()
        
    violated_LP = 0
    #Check for violated constraints
    for i in Settlement_List:
        for j in District_List:
            for k in Appeals_List:
                #Assign vars based on 0 or gurobi.Var               
                d_val = VarVal(d_LP[i][j])
                a_val = VarVal(a_LP[i][k])
                c_val = VarVal(c_LP[j][k])
                
                #Add constraints if violated
                if (d_val + a_val - c_val > 1):
                    print(i,j,k)
                    print(d_val,a_val,c_val)
                    violated_LP = violated_LP + 1
                    LP.addConstr(d_LP[i][j] + a_LP[i][k] - c_LP[j][k] <= 1)
    LP.update()


LP Optimize
43035 3002.0 30.0
1.0 1.0 0.0
31430 1707.0 17.0
1.0 1.0 0.0
14993 2707.0 18.0
1.0 0.5 0.0
14993 2707.0 27.0
1.0 0.5 0.0
21045 2001.0 20.0
1.0 0.5 0.0
21045 2001.0 27.0
1.0 0.5 0.0
25357 1816.0 18.0
1.0 0.5 0.0
25357 1816.0 20.0
1.0 0.5 0.0
43002 3002.0 30.0
1.0 1.0 0.0
28515 618.0 6.0
1.0 1.0 0.0
16248 2110.0 31.0
1.0 1.0 0.0
19185 1110.0 11.0
1.0 1.0 0.0
31270 1007.0 10.0
1.0 1.0 0.0
20937 2003.0 18.0
1.0 0.5 0.0
20937 2003.0 20.0
1.0 0.5 0.0
35121 910.0 17.0
1.0 1.0 0.0
4117 3016.0 30.0
1.0 1.0 0.0
4681 3210.0 32.0
1.0 1.0 0.0
17635 2101.0 21.0
1.0 1.0 0.0
27434 604.0 6.0
1.0 1.0 0.0
18017 2207.0 21.0
1.0 1.0 0.0
5258 3109.0 32.0
1.0 1.0 0.0
31145 1112.0 10.0
1.0 1.0 0.0
39084 301.0 3.0
1.0 1.0 0.0
6990 3002.0 30.0
1.0 1.0 0.0
15400 3016.0 21.0
1.0 1.0 0.0
12358 2202.0 22.0
1.0 1.0 0.0
21862 2501.0 12.0
1.0 1.0 0.0
23715 2605.0 12.0
1.0 1.0 0.0
30137 1707.0 17.0
1.0 1.0 0.0
41871 301.0 3.0
1.0 1.0 0.0
31660 1007.0 10.0
1.0 1.0 0.0
23065 2605.0 12.0
1.0 1.0 0.0
41657 805.0

KeyboardInterrupt: 

In [ ]:
d_LP

### Output Solution

In [ ]:
LP.write("out_LP.sol")
MIP.write("out_MIP.sol")

###  Plotting

#### DOUT

In [ ]:
# DOUT - District Assignments

DOUT = pd.DataFrame.from_dict({(i,j): VarVal(d[i][j]) 
                           for i in d.keys() 
                           for j in d[i].keys()},
                           orient='index')

Settlements_DOUT = [i[0] for i in DOUT.index]
Districts_DOUT = [i[1] for i in DOUT.index]

DOUT['Settlement'] = Settlements_DOUT
DOUT['DistrictCourts'] = Districts_DOUT

DOUT = DOUT[DOUT[0] == 1.0]

#### AOUT

In [ ]:
#AOUT - Appeals Assignments

AOUT = pd.DataFrame.from_dict({(i,j): VarVal(a[i][j]) 
                           for i in a.keys() 
                           for j in a[i].keys()},
                           orient='index')

Settlements_AOUT = [i[0] for i in AOUT.index]
Appeals_AOUT = [i[1] for i in AOUT.index]

AOUT['Settlement'] = Settlements_AOUT
AOUT['AppealsCourts'] = Appeals_AOUT

AOUT = AOUT[AOUT[0] == 1.0]

#### COUT

In [ ]:
#COUT - Appeals Assignments

COUT = pd.DataFrame.from_dict({(i,j): VarVal(c[i][j]) 
                           for i in c.keys() 
                           for j in c[i].keys()},
                           orient='index')

Districts_COUT = [i[0] for i in COUT.index]
Appeals_COUT = [i[1] for i in COUT.index]

COUT['DistrictCourts'] = Districts_COUT
COUT['AppealsCourts'] = Appeals_COUT

COUT = COUT[COUT[0] == 1.0]

#### Plot all Locations

In [ ]:
plt.scatter(Settlements['LON_X'],Settlements['LAT_Y'])
plt.scatter(Districts['LON_X'],Districts['LAT_Y'], color='Red', marker = 's')
plt.scatter(Appeals['LON_X'],Appeals['LAT_Y'], color='Green', marker = '^')
plt.show()

#### Plotting Assignments

In [ ]:
#Drawing Lines
plt.figure(figsize=(14,14))

#Plotting Points    
plt.scatter(Settlements['LON_X'],Settlements['LAT_Y'])
plt.scatter(Districts['LON_X'],Districts['LAT_Y'], color='Red', marker = 's',s=25)
plt.scatter(Appeals['LON_X'],Appeals['LAT_Y'], color='Green', marker = '^',s = 400)

for index,row in DOUT.iterrows():
    s = row['Settlement'] 
    d = row['DistrictCourts']
    #Get District number that Settlement is linked to
    Dist = Districts.loc[Districts['DIST_CODE'] == d]
    Sett = Settlements.loc[Settlements['OBJECTID'] == s]
    X = [Sett.iloc[0,2],Dist.iloc[0,1]]
    Y = [Sett.iloc[0,1],Dist.iloc[0,2]]
    plt.scatter(Dist.iloc[0,1],Dist.iloc[0,2], color='Orange', marker = 's',s = 100)
    plt.plot(X,Y,zorder=1, color="Black")



#axes = plt.gca()
#axes.set_xlim([68.2,70])
#axes.set_ylim([34,35.5])

plt.show()

In [ ]:
#Draw Lines
plt.figure(figsize=(14,14))

for index,row in COUT.iterrows():
    d = row['DistrictCourts'] 
    a = row['AppealsCourts']

    Dist = Districts.loc[Districts['DIST_CODE'] == d]
    App = Appeals.loc[Appeals['PROV_CODE'] == a]

    X = [Dist.iloc[0,1],App.iloc[0,1]]
    Y = [Dist.iloc[0,2],App.iloc[0,2]]

    plt.plot(X,Y,zorder=1, color="Black")
    
clr = cm.rainbow(np.linspace(0, 1, D))
for a in range(A):
    X = Appeals.iloc[a,1]
    Y = Appeals.iloc[a,2]
    plt.scatter(X,Y, color="Green", marker = '^')
    
for index,row in DOUT.iterrows():   
    d = row['DistrictCourts'] 
    Dist = Districts.loc[Districts['DIST_CODE'] == d]
    X = Dist.iloc[0,1]
    Y = Dist.iloc[0,2]
    c = Dist.index[0]
    plt.scatter(X,Y, color=clr[c], marker = 's')
    
for index,row in DOUT.iterrows():
    s = row['Settlement'] 
    d = row['DistrictCourts']
    #Get District number that Settlement is linked to
    Dist = Districts.loc[Districts['DIST_CODE'] == d]
    Sett = Settlements.loc[Settlements['OBJECTID'] == s]

    c = Dist.index[0]
    X = Sett.iloc[0,2]                                                     
    Y = Sett.iloc[0,1]
    plt.scatter(X,Y, color=clr[c])
          
#axes = plt.gca()
#axes.set_xlim([68.2,70])
#axes.set_ylim([34,35.5])

sf = shp.Reader("Afghanistan_Districts","rb")
for shape in sf.shapeRecords():
    x = [i[0] for i in shape.shape.points[:]]
    y = [i[1] for i in shape.shape.points[:]]
    plt.plot(x,y,color='k',linewidth=0.1)
plt.show()